# Reporting Conditions and Predicted Capacities

In [ ]:
import pandas as pd
import matplotlib as plt
import captest as pvc

First we load the NREL data used for testing and set the translation dictionary.

In [ ]:
meas = pvc.CapData('meas')

In [ ]:
meas.load_data(path='../tests/data/', fname='nrel_data.csv', source='AlsoEnergy')

In [ ]:
meas.trans

In [ ]:
meas.set_reg_trans(poa='irr-poa-', w_vel='wind--', t_amb='temp--', power='')

In [ ]:
meas.filter_irr(500, 600)

Below is an exmaple of calculating reporting conditions using all defaults, which are set based on the ASTM E2939 standard.  The reporting condition method always performs calculations using the filtered dataframe stored in the df_flt attribute.

- RC_irr 60% percentile
- RC_temp mean
- RC_wind mean
- Calculates values on filtered data
- inplace

In [ ]:
meas.rep_cond()

Use all defaults, except inplace
- RC_irr 60% percentile
- RC_temp mean
- RC_wind mean
- Calculates values on filtered data
- not inplace- returns rep cond and doesn't write to RCs

In [ ]:
meas.rc

In [ ]:
meas.rc = None

In [ ]:
meas.rep_cond(inplace=False)

In [ ]:
print(meas.rc)

## Irradiance Balance Method
The irr_bal option replaces the aggregation methods specified with the func agrument with the irrRC_balanced function.  This function is intended to implement a strict interpratation of common contract language that specifies the reporting irradiance be determined by finding the irradiance that results in a balance of points within a +/- percent range of the reporting irradiance.  The irrRC_balanced function iterates to a solution for the reporting irradiance by calculating the irradiance that has 10 datpoints in the filtered dataset above it, then filtering for a percentage of points around that irradiance, calculating what percentile the reporting irradiance is in 

In [ ]:
meas.reset_flt()

In [ ]:
meas.filter_irr(500, 900)

The plot below shows a visualization of the irrRC_balanced calculations. The teal dots in a vertical column along the left side of the figure are all the irradiance measurements in the dataset. Each red dot is an irradiance reporting condition and the colored vertical lines are the extent of the percent band around the reportin irradiance.  The reportign irradiance is decreased for each loop until 40% of the irradiance points are above the reporting irradiance.

In [ ]:
rep_irr = pvc.irrRC_balanced(meas.df_flt, 0.8, 1.2, irr_col=meas._CapData__get_poa_col(), plot=True)

In [ ]:
rep_irr[0]

In [ ]:
meas.df_flt.shape[0]

In [ ]:
rep_irr[1].shape[0]

In [ ]:
meas.rep_cond(irr_bal=True, perc_flt=20)

In [ ]:
meas.rc

## Reporting Conditions spanning New Year Using TMY Data

The cntg_eoy year function can be used to wrap tmy data around the beginning/end of the year to prepare for calculating reporting conditions using time periods that span the new year.

In [ ]:
pvsyst = pvc.CapData('pvsyst')

In [ ]:
pvsyst.load_data(load_pvsyst=True)

In [ ]:
pvsyst.trans

In [ ]:
pvsyst.set_reg_trans(power='real_pwr--', poa='irr-poa-', t_amb='temp-amb-', w_vel='wind--')

In [ ]:
pvsyst.df_flt = pvc.cntg_eoy(pvsyst.df_flt, 
                             pd.Timestamp(year=1989, month=12, day=15),
                             pd.Timestamp(year=1990, month=1, day=15))

In [ ]:
pvsyst.df_flt.head(2)

In [ ]:
pvsyst.df_flt.tail(2)

In [ ]:
pvsyst.filter_irr(200, 900)

In [ ]:
pvsyst.filter_pvsyst()

In [ ]:
pvsyst.rep_cond()

## Reporting Conditions for Time Periods

The freq option can be used to calculate reporting conditions for groups of time like months or seasons.  Any standard aggregation method can be used to calculate the reporting conditions for each group using the func or the irr_bal method options.

In [ ]:
pvsyst.reset_flt()

In [ ]:
print(pvsyst.df_flt.index[0])
print(pvsyst.df_flt.index[-1])

In [ ]:
pvsyst.filter_irr(500, 900)

In [ ]:
pvsyst.rep_cond(freq='MS')

In [ ]:
pvsyst.rc

Using the irr_bal option to calculate reporting conditions for each month of the year results in higher reporting irradiances.

Note that the with inplace set to False the reporting conditions attribute of the CapData object, pvsyst, is not overwritten.

In [ ]:
ib_rcs = pvsyst.rep_cond(freq='MS', irr_bal=True, perc_flt=20, inplace=False)
ib_rcs

In [ ]:
(ib_rcs - pvsyst.rc)['poa']

## Predicted Capacities
The `predict_capacities` method will use reporting conditions calculated using the freq option to determine predicted and guaranteed capacities.

From the previous exercise we still have the montly reporting conditions.

In [ ]:
pvsyst.rc

In [ ]:
pvsyst.tolerance = '+/- 7'

In [ ]:
pvsyst.predict_capacities()